In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
import csv
import datetime
import re

In [2]:
# my_date = datetime.date.today() 
# year, week_num, day_of_week = my_date.isocalendar()
# this_week = week_num

# url_for_scrape = 'https://www.metacritic.com/browse/albums/release-date/new-releases/date'
# user_agent = {'User-agent': 'Mozilla/5.0'}
# # send response
# response_score = requests.get(url_for_scrape, headers = user_agent)
# # scrape website into variable to parse
# soup_score = BeautifulSoup(response_score.text, 'html.parser')

# # create temporary lists for user scores
# userP = []
# userM = []
# userN = []
# # create/initialize dictionary 
# albums_dict = {'artist':[], 'album':[], 'date':[], 'week_num':[], 'meta_score': [], 'user_score':[]}

# soup_score.find_all('td', class_='clamp-summary-wrap')
# # create soup 
# for _ in soup_score.find_all('td', class_='clamp-summary-wrap'):
#     # scrape album name
#     albums_dict['album'].append(_.find('a', class_= 'title').text)
#     # scrape artist name and strip white space and extra characters
#     albums_dict['artist'].append(_.find('div', class_='artist').text.strip().lstrip('by '))
#     # scrape date
#     albums_dict['date'].append(_.find('div', class_='clamp-details').find('span').text)
#     # scrape meta_score, handle for changes in class name, convert data type of score to int and append to dict
#     # except set to pass since all alubms have a score

#     try:
#         albums_dict['meta_score'].append(int(_.find('div', class_='metascore_w large release positive').text))  
#     except:
#         pass
#     try:
#         albums_dict['meta_score'].append(int(_.find('div', class_='metascore_w large release mixed').text))  
#     except:
#         pass 
#     try:
#         albums_dict['meta_score'].append(int(_.find('div', class_='metascore_w large release negative').text))  
#     except:
#         pass
#     # scrape user score, handle errors for tbd/class name and append to temp list
#     try:
#         userP.append(float(_.find('div', class_='metascore_w user large release positive').text))  
#     except:
#         userP.append(0)
#     try:
#         userM.append(float(_.find('div', class_='metascore_w user large release mixed').text))  
#     except:
#         userM.append(0)
#     try:
#         userN.append(float(_.find('div', class_='metascore_w user large release negative').text))  
#     except:
#         userN.append(0)

# # merge user score by filtering scores from tbd using data type in temporary lists, convert data type of scores to int and append to dictionary        
# for a, b, c in zip(userP, userM, userN):
#     if isinstance(a, float):
#         albums_dict['user_score'].append(int(a * 10))
#     elif isinstance(b, float):
#         albums_dict['user_score'].append(int(b * 10))
#     elif isinstance(c, float):
#         albums_dict['user_score'].append(int(c * 10))
#     else:
#         albums_dict['user_score'].append(c)
# # create week_num key and values for weekly scrape
# for dates in albums_dict['date']:
#     albums_dict['week_num'].append((datetime.datetime.strptime(dates, '%B %d, %Y')).isocalendar()[1])
# # write dictionary to csv
# # create header
# fields = ['artist', 'album', 'date', 'week_num', 'meta_score', 'user_score'] 
# # create variable for data to be written
# data = zip(albums_dict['artist'], albums_dict['album'], albums_dict['date'], albums_dict['week_num'], albums_dict['meta_score'], albums_dict['user_score'])
# output_path = os.path.join('..', 'data', 'meta_scrape.csv')
# # create header
# fields = ['artist', 'album', 'date', 'week_num', 'meta_score', 'user_score'] 
# # create variable for data to be written
# data = zip(
#         albums_dict['artist'], 
#         albums_dict['album'], 
#         albums_dict['date'], 
#         albums_dict['week_num'], 
#         albums_dict['meta_score'], 
#         albums_dict['user_score']
#         )
# with open(output_path, 'a') as csvfile:
#     writer = csv.writer(csvfile)
#     for d in data:
#         writer.writerow(d)



In [3]:
file_path = os.path.join('..', 'data', 'meta_scrape.csv')

In [4]:
scrape_df = pd.read_csv(file_path)  
# dedup df        
scrape_clean = scrape_df.drop_duplicates(subset = 'artist', ignore_index = True)


In [5]:
scrape_clean.loc[scrape_clean['album'] == 'Local Valley']


,artist,album,date,week_num,meta_score,user_score
59,José González,Local Valley,"September 17, 2021",37,76,0


In [22]:
scrape_clean

,artist,album,date,week_num,meta_score,user_score
0,Pokey LaFarge,In the Blossom of Their Shade,"October 15, 2021",41,85,0
1,Hayden Thorpe,Moondust for My Diamond,"October 15, 2021",41,77,0
2,Coldplay,Music of the Spheres,"October 15, 2021",41,57,64\
3,FINNEAS,Optimist,"October 15, 2021",41,71,89
4,Remi Wolf,Juno,"October 15, 2021",41,91,88
...,...,...,...,...,...,...
155,audiobooks,Astro Tough,"October 1, 2021",39,73,0
156,Ministry,Moral Hygiene,"October 1, 2021",39,71,0
157,Black Dice,Mod Prog Sic,"October 1, 2021",39,78,0
158,Boy Scouts,Wayfinder,"October 1, 2021",39,74,0


In [36]:
import time

reviews =[]
def create_review_url(df):
    review_urls = []
    # iterate over dataframe
    for i, j in scrape_clean.iterrows():
        ar = j['artist']
    #     clean album and artist names of all nonalpha except for ! and accent marks
        al = re.sub(r'[^-A-Za-z0-9!áéíóúÁÉÍÓÚâêîôÂÊÎÔãõÃÕçÇ ]+', '', j['album'])
        ar = re.sub(r'[^-A-Za-z0-9!áéíóúÁÉÍÓÚâêîôÂÊÎÔãõÃÕçÇ ]+', '', j['artist'])
    # correct instances of multilple spaces
        al = re.sub(' +', ' ', al)
        ar = re.sub(' +', ' ', ar)
    # make lowercase and join words with - for url
        url_end = f'{al}/{ar}'.replace(" ", "-").lower() 
    # concat url for review scrape    
        url_beginning ='https://www.metacritic.com/music/'
        review_urls.append(url_beginning + url_end)
    #     print(requests.get(url_review_scrape, headers = user_agent))

#     user_agent = {'User-agent': 'Mozilla/5.0'}
#     review_response = 
    return review_urls

In [37]:
create_review_url(scrape_clean)

['https://www.metacritic.com/music/in-the-blossom-of-their-shade/pokey-lafarge',
 'https://www.metacritic.com/music/moondust-for-my-diamond/hayden-thorpe',
 'https://www.metacritic.com/music/music-of-the-spheres/coldplay',
 'https://www.metacritic.com/music/optimist/finneas',
 'https://www.metacritic.com/music/juno/remi-wolf',
 'https://www.metacritic.com/music/sticky/frank-carter-the-rattlesnakes',
 'https://www.metacritic.com/music/to-hell-with-it-mixtape/pinkpantheress',
 'https://www.metacritic.com/music/skin/joy-crookes',
 'https://www.metacritic.com/music/let-it-be-2021-mix-super-deluxe-edition-box-set/the-beatles',
 'https://www.metacritic.com/music/the-atlas-underground-fire/tom-morello',
 'https://www.metacritic.com/music/lately/lilly-hiatt',
 'https://www.metacritic.com/music/georgia-blue/jason-isbell-and-the-400-unit',
 'https://www.metacritic.com/music/talk-memory/badbadnotgood',
 'https://www.metacritic.com/music/seventeen-going-under/sam-fender',
 'https://www.metacritic.

TypeError: object of type 'int' has no len()

In [ ]:
artist = {}
    url_for_reviews = f'https://www.metacritic.com/music/the-myth-of-the-happily-ever-after/{artist}'
    user_agent = {'User-agent': 'Mozilla/5.0'}
    # send response
    response_reviews = requests.get(url_for_reviews, headers = user_agent)
    # scrape website into variable to parse
    soup_reviews = BeautifulSoup(response_reviews.text, 'html.parser')
    img_soup = soup_reviews.find('meta', property="og:image")
    artist['image'] = img_soup['content']
    num_rev=soup_reviews.find('span', itemprop="reviewCount")
    artist['crit_rev_num'] = num_rev.text.strip()
    user_rev = soup_reviews.find('span', class_='count')
    print(user_rev)

In [ ]:
#     artist['user_rev_num'] = 
#     artist['critic_rev_num']
    break
#     for link in soup_reviews.find_all('div', class_='product_image large_image'):
#         print(link.meta.descendents)

In [ ]:
# /html/body/div[1]/div[2]/div[1]/div[1]/div/div/div/div/div/div/div/div/div[1]/div[1]/div[3]/div/div[2]/div[2]/div[2]/ul/li[1]/span[1]

In [ ]:
# <img alt="The Myth of the Happily Ever After Image" class="product_image large_image" src="https://static.metacritic.com/images/products/music/8/3f66ba56e8190a1a9f04fb37bdd986e8-98.jpg"/>




In [ ]:
# Making a Connection with MongoClient
client = MongoClient("mongodb://localhost:27018/")
# database
db = client["stocks_database"]
# collection
company= db["Company"]